# Lab 3 - Using UDP and Protobuf

The aim of this lab is to use UDP sockets and google protobuf to implement a simple satellite ground station system capable of sending command requests to a satellite and parsing the replies. This lab will introduce the usage of protobufs and reinforce your understanding of UDP sockets. These skills will assist you in our next lab which will return to the file transfer system created in Lab 2.

You will need to install the protobuf plugin for PyCharm. Settings > Plugins > Protobuf.

There are two executables included with this lab that you will need. A protobuf compiler and the satellite control server.

## 3.1 - Primer
Before beginning this lab it is highly recommended that you review the protobuf lecture slides (you won't need to manually encode the data for this lab). You may also want to reference this [protobuf python tutorial](https://protobuf.dev/getting-started/pythontutorial/) and the [proto2 language guide](https://protobuf.dev/programming-guides/proto2/).

The purpose of this lab is to build a ground station to communicate with a (simulated) satellite orbiting in space.

Due to the relevant frequency allocations and normal levels of noise from cosmic background radiation, the TCP protocol is considered unsuitable. When previously tested by (also simulated) satellite engineers, TCP resulted in too many broken connections. As such, you will need to use UDP to communicate with the satellite.

The satellite is already in orbit and supports the following commands:

- Get the satellite's current location;
- Get current time (from the satellite's internal clock);
- Move the satellite.

The satellite is already configured to reply to these commands using UDP. As UDP does not guarantee message delivery, commands sent to the satellite and/or replies from the satellite may be lost. To accommodate potentially lost messages your ground station system must utilize timeouts and attempt to retry communications 5 times before giving up and waiting for the next command. Your system must return control to the user if no response is heard from the satellite within 3 seconds.

Note: the server executable is almost always in a blocking state waiting either for keyboard input or messages from the ground station. As a result, `ctl-c` will not exit the program and you need to close the terminal if you wish to reset the server.

## 3.2 - Satellite Message Types
Python 3 integers are all arbitrary precision (they grow in memory as needed to accommodate their values) and Python 3 floats are all double precision (64-bit). Protobufs support multiple kinds of values but will convert to Python's format when parsing received data. In order to ensure interoperability with other languages, you need to match the definitions for the sender and receiver. The value types given in the tables below line up to the value types available within the protobuf documentation's scalar value table. The exact protobuf types are not given and it is expected that you will use a combination of testing and analysis to ensure you choose the correct value types from the suitable options.

### Ground Station to Satellite Request
All ground station to satellite requests use the same variable message format.
Note: tables may be easier to view as raw text rather than formatted markdown.

#### A - Mandatory data fields:

| Type        | Purpose              | Identifier |
|-------------|----------------------|------------|
| Integer     | Satellite Identifier | 1          |
| RequestType | The Type of Request  | 2          |

#### B - Optional Fields for Satellite Movement Command
Note: Mandatory for movement commands but should not be present for other commands.

| Type   | Purpose                      | Identifier |
|--------|------------------------------|------------|
| Double | Angle To Fire Thrusters      | 3          |
| Double | Duration To Fire Thrusters   | 4          |

RequestType is a special enumerated type. RequestType must support the following values:

| Request Type     | Value |
|------------------|-------|
| Location Request | 0     |
| Time Request     | 1     |
| Move Request     | 2     |

### Satellite Responses

#### A - Location Request Response
Note: For expandability all fields are optional.

| Type    | Purpose    | 	Identifier |
|---------|------------|-------------|
| Double  | 	Latitude  | 	1          |
| Double  | 	Longitude | 	2          |

#### B - Time Request Response

| Type | Purpose            | 	Identifier |
|------|--------------------|-------------|
| Long | Current Time in ms | 1           |

#### C - Move Request Response

| Type             | Purpose                      | 	Identifier |
|------------------|------------------------------|-------------|
| Integer          | Response Code                | 1           |
| LocationResponse | Updated Location Information | 2           |

A LocationResponse is the same as the message defined for "Location Request Response"

## 3.3 - Implementation

Using the definitions above, create a protocol buffer file to define the messages necessary for your ground station system to communicate to the satellite. The satellite binds to UDP port 4444. When a command is issued from the user, your program will need to craft the proper message using your protobuf definition and send it to the server. The server will parse the request and send back a reply.

Your ground station system must:

- Provide a simple command line interface to enable the user to send commands to a satellite;
    - A text prompt with instructions on issuing commands is sufficient but feel free to pursue more creative solutions;
- Enable the user to issue time, location and move requests to a satellite;
    - The user must be able to specify values for all fields relevant to each message type;
- Reattempt communications if a response isn't received;
    - Reattempts are to continue until a response is received or 5 attempts fail;
- Never get stuck waiting for a response for more than 3 seconds (3 seconds total for all attempts and reattempts);
- Enable the user to quit the application;
- Parse all received replies;
- Display appropriate information to the user when replies are received from a satellite:
    - For Time Request responses the ground station system must display the current date and time provided by the satellite in the following format: Day Month Year HH:MM:SS;
    - For Location Request responses the ground station system must display the names and values of each field contained in the response;
    - For Move Request responses the ground station system must provide the user with different information depending on the response code provided:
        - A response code of 0 indicates a failure and the user must be notified;
        - A response code of 1 indicates success and the new location information provided must be displayed to the user;

### Starting Out
The satellite control server is provided as an executable file for you to interface with. It is included in the lab files as `.\dist\satellite_server\satellite_server.exe`. It has been packaged for Windows use only.

A python protobuf compiler is provided for you to convert your `.proto` message format into python classes. You will need to use the command line, either in an Anaconda prompt or with the integrated terminal in your IDE. The following command will compile your message format into python code in your local directory:
```
# generate ground station stub
.\protoc\bin\protoc.exe satellite.proto --python_out=.
```
Note that this code doesn't have a class structure. It is helpful to understand how your message format is parsed into python so run:
```
# generate class descriptions
.\protoc\bin\protoc.exe satellite.proto --pyi_out=.
```
The `.pyi` file cannot be used to import classes. It is the human-readable version of the `.py` stub interface.

Hint: One way to approach this task is to create a ground station class which implements the required methods. Your ground station needs to send a protobuf-formatted message via UDP and receive a response from the server. It may be helpful to sketch out your intended implementation on paper or as skeleton methods before proceeding with building out your code.

### Troubleshooting

You may encounter difficulties debugging the serialization and deserialization of your data. Note that you can access the hexadecimal representation of a string of bytes in python 3 using its hex method.

```python
aByteString = b'hello'
print(aByteString.hex())
#output is 68656c6c6f
```

## 3.4 - Submission
Your objective for this lab is to implement the ground station as described in section 3.3. The set of deliverables includes:
    1. Python files containing your ground station code and modules (at least one `.py` script file and your `_pb2.py` file).
    2. `.proto` file;
    3. Lab report notebook.

All files included in this lab (i.e., your project) are to be submitted in a zip file (see lab_report_template.ipynb for instructions). Use the lab report template to reference the files that you created. There are no questions for this lab.